In [19]:
import pandas as pd


In [20]:
df = pd.read_csv('/content/res_1.csv')


<ipython-input-20-c2a1bbc9e2fa>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/res_1.csv')


In [21]:
# Find duplicates
duplicate_rows = df[df.duplicated(keep=False)]

print('before deleting duplicates: ' ,df.shape)
df = df.drop_duplicates(keep='first')

print('after deleting duplicates: ', df.shape)

before deleting duplicates:  (90263, 19)
after deleting duplicates:  (89844, 19)


In [22]:
# Drop the specified columns
df = df.drop(columns=['SNumber', 'YPN1', 'YPN2', 'ActionStatus'])

# Replace null values in the ypn2_ID column with 0
df['ypn2_ID'] = df['ypn2_ID'].fillna(0)

In [23]:
# Filter the DataFrame to show only rows where ActionStatusDate is null
null_dates = df[df['OrderDate'].isna()]



In [24]:
# Drop the rows where ActionStatusDate is null in place
df.dropna(subset=['OrderDate'], inplace=True)
# Extract the numeric part from the Shift column
df['Shift'] = df['Shift'].str.extract(r'(\d+)').astype(int)
df['Machines_Zone'] = df['Machines_Zone'].str.extract(r'(\d+)').astype(int)
# Replace 'sw' with 0 and 'dw' with 1 in the Capability column
df['Capability'] = df['Capability'].replace({'SW': 0, 'DW': 1})
# Convert Shiftdate to datetime
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Extract day, month, and hour from Shiftdate
df['Year'] = df['OrderDate'].dt.year
df['Day'] = df['OrderDate'].dt.day
df['Month'] = df['OrderDate'].dt.month
df['Hour'] = df['OrderDate'].dt.hour
df['Minute'] = df['OrderDate'].dt.minute
df = df.drop(columns=['Shiftdate'])
df['ypn1_ID'] = df['ypn1_ID'].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89844 entries, 0 to 90262
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Seq            89844 non-null  int64         
 1   Urgent         89844 non-null  int64         
 2   CAONO          89844 non-null  int64         
 3   Quantity       89844 non-null  object        
 4   Shift          89844 non-null  int64         
 5   ProdProgress   89844 non-null  int64         
 6   Machines_ID    89844 non-null  int64         
 7   Machines_Zone  89844 non-null  int64         
 8   Capability     89844 non-null  int64         
 9   OrderDate      89844 non-null  datetime64[ns]
 10  isFilAvail     89844 non-null  int64         
 11  S_number_id    89844 non-null  int64         
 12  ypn1_ID        89844 non-null  float64       
 13  ypn2_ID        89844 non-null  float64       
 14  Year           89844 non-null  int32         
 15  Day            89844 non

In [25]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df = df[df['Quantity'] >= 5]
df = df[df['Quantity'] <=100000 ]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89771 entries, 0 to 90262
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Seq            89771 non-null  int64         
 1   Urgent         89771 non-null  int64         
 2   CAONO          89771 non-null  int64         
 3   Quantity       89771 non-null  float64       
 4   Shift          89771 non-null  int64         
 5   ProdProgress   89771 non-null  int64         
 6   Machines_ID    89771 non-null  int64         
 7   Machines_Zone  89771 non-null  int64         
 8   Capability     89771 non-null  int64         
 9   OrderDate      89771 non-null  datetime64[ns]
 10  isFilAvail     89771 non-null  int64         
 11  S_number_id    89771 non-null  int64         
 12  ypn1_ID        89771 non-null  float64       
 13  ypn2_ID        89771 non-null  float64       
 14  Year           89771 non-null  int32         
 15  Day            89771 non

In [27]:
## Check if each CAONO has a unique S_number_id
unique_s_number_per_caono = df.groupby('CAONO')['S_number_id'].nunique()
if (unique_s_number_per_caono == 1).all():
    print("Each CAONO has a unique S_number_id.")
else:
    print("There are CAONO values with multiple S_number_id values.")

Each CAONO has a unique S_number_id.


In [28]:
caono_counts = df.groupby('S_number_id')['CAONO'].nunique()
print((caono_counts>1)==True)

S_number_id
1        False
2        False
3        False
4        False
5        False
         ...  
10474    False
10475    False
10476    False
10477    False
10478    False
Name: CAONO, Length: 10473, dtype: bool


In [29]:
# Count of CAONO per S_number_id
caono_counts = df.groupby('S_number_id')['CAONO'].nunique()
print(caono_counts)

S_number_id
1        1
2        1
3        1
4        1
5        1
        ..
10474    1
10475    1
10476    1
10477    1
10478    1
Name: CAONO, Length: 10473, dtype: int64


In [30]:

# Get the S_number_id values that have more than one CAONO
s_numbers_with_multiple_caono = caono_counts[caono_counts > 1].index
print(s_numbers_with_multiple_caono)

Index([   12,    15,    17,    18,    20,    21,    22,    24,    25,    26,
       ...
       10437, 10438, 10439, 10440, 10441, 10442, 10443, 10449, 10452, 10463],
      dtype='int64', name='S_number_id', length=7645)


In [31]:
# Convert action_statusdate to datetime
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# Sort the DataFrame by CAONO and action_statusdate
df = df.sort_values(by=['CAONO','OrderDate'])

# Add a new column that ranks the steps of a distinct CAONO based on action_statusdate
df['step_rank'] = df.groupby('CAONO')['OrderDate'].rank(method='first', ascending=True)


In [33]:
df.to_csv('final_ymt_2.csv', index=False)


In [32]:
print(df)


       Seq  Urgent     CAONO  Quantity  Shift  ProdProgress  Machines_ID  \
46706    3       0  52084949      10.0      2           300          412   
46707    2       0  52084949      10.0      2           300          412   
46836    1       0  52084949      10.0      2           400          412   
1302     1       0  52233634    2500.0      3           300          488   
1305     2       0  52233634    2500.0      3           300          488   
...    ...     ...       ...       ...    ...           ...          ...   
90167    1       1  52329542     100.0      2           300          236   
90176    2       1  52329542     100.0      2           300          236   
90224    2       1  52329544      50.0      2           300          310   
90241    1       1  52329544      50.0      2           400          310   
90246    3       1  52329551      50.0      2           300          310   

       Machines_Zone  Capability               OrderDate  isFilAvail  \
46706          